Data Cleaning and preprocessing

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Importing the dataset in csv format into pandas dataframe
df = pd.read_csv("database/final_crime_data.csv")

In [ ]:
df.shape

Print the first five rows of the dataframe

In [ ]:
df.head()

In [ ]:
import pandas as pd

df1 = pd.read_csv('Database/final/crime_data_modified_final.csv')

# Load the second file
df2 = pd.read_csv('Database/final/crime_data_modified_2018_removed_final.csv')



# Append df2 to df1
combined_df = pd.concat([df1, df2], ignore_index=True)

# Save the combined dataframe back to a new CSV file or overwrite one of the originals
combined_df.to_csv('Database/final/combined_file.csv', index=False)


Removing the columns cr_cd_4, cross_street, cr_cd_3, cr_cd_2

In [ ]:
import pandas as pd
import plotly.express as px

import pandas as pd

# Load the dataset
df = pd.read_csv('Database/final/crime_data_modified_2018_removed.csv')

# Convert 'date_occ' to datetime format if it's not already
df['date_occ'] = pd.to_datetime(df['date_occ'], errors='coerce')

# Extract the year and create a new column 'year_occ'
df['year_occ'] = df['date_occ'].dt.year

# Save the modified dataframe back to a csv file if needed
df.to_csv('Database/final/crime_data_modified_2018_removed_final.csv', index=False)

In [ ]:
import pandas as pd

# Assuming df is your DataFrame and it's already loaded with your data
df = pd.read_csv('Database/final/crime_data_modified.csv')  # Replace 'your_dataset.csv' with your actual file path

# Filter the DataFrame to include only rows where 'year_rept' is 2018 or 2019
filtered_df = df[df['year_rept'].isin([2018, 2019])]

# Now filtered_df contains only the rows from the years 2018 and 2019
filtered_df.to_csv('Database/final/crime_data_modified1.csv')

In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv('crime_data_modified_2018.csv')
# Removing rows where the year column is 2016
# List of years to be removed
years_to_remove = [2010, 2011, 2012, 2013, 2014]

# Removing rows where the year column is in the list of years to remove
df = df[~df['year_rept'].isin(years_to_remove)]

# Display the DataFrame after removal
print("\nDataFrame after removing rows for years 2011, 2012, 2013, and 2014:")
print(df)

df.to_csv('crime_data_modified_2018_removed.csv')

In [ ]:
import pandas as pd

# Load the first file
df1 = pd.read_csv('crime_data_modified.csv')

# Load the second file
df2 = pd.read_csv('crime_data_modified_2018_removed.csv')

# Append df2 to df1
combined_df = pd.concat([df1, df2], ignore_index=True)

# Save the combined dataframe to a new CSV file
combined_df.to_csv('combined_file.csv', index=False)

In [ ]:
# Load the first file
# Load the first file
df1 = pd.read_csv('Database/final/crime_data_modified_final.csv')

# Load the second file
df2 = pd.read_csv('Database/final/crime_data_modified_2018_removed_final.csv')

# Filter out rows where latitude or longitude is 0
df1 = df1[(df1['lat'] != 0) & (df1['lon'] != 0)]
df2 = df2[(df2['lat'] != 0) & (df2['lon'] != 0)]

df1.to_csv('Database/final/crime_data_modified_final.csv', index=False)
df2.to_csv('Database/final/crime_data_modified_2018_removed_final.csv', index=False)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load data from CSV files
data_2018_2019 = pd.read_csv('Database/final/crime_data_modified_2018_removed_final.csv')
data_2020_2024 = pd.read_csv('Database/final/crime_data_modified_final.csv')

# Combine the two dataframes
data_combined = pd.concat([data_2018_2019, data_2020_2024], ignore_index=True)

# Group the data by year and area, and count the occurrences
crime_counts = data_combined.groupby(['year_rept', 'area_name']).size().reset_index(name='count')

# Calculate the total crimes per year to use for normalization
total_crimes_per_year = crime_counts.groupby('year_rept')['count'].sum()

# Merge the total crimes back with the crime counts to calculate normalized rates
normalized_crime_rates = crime_counts.merge(total_crimes_per_year, on='year_rept', suffixes=('', '_total'))
normalized_crime_rates['normalized_rate'] = normalized_crime_rates['count'] / normalized_crime_rates['count_total']

# Pivot data for plotting
pivot_table = normalized_crime_rates.pivot(index='area_name', columns='year_rept', values='normalized_rate')

# Plotting
pivot_table.plot(kind='bar', figsize=(14, 7))
plt.title('Normalized Crime Rates by Area and Year')
plt.ylabel('Normalized Crime Rate')
plt.xlabel('Area Name')
plt.xticks(rotation=45)
plt.legend(title='Year')
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import folium
import ipywidgets as widgets
from IPython.display import display

# Load the crime data
df = pd.read_csv("Database/final/crime_data_modified.csv")

# Filter unique years
unique_years = df['year_rept'].unique()

# Dropdown widget for selecting years
year_dropdown = widgets.Dropdown(
    options=unique_years,
    value=unique_years[0],
    description='Year:'
)

# Function to update the map based on selected year
def update_map(year):
    # Filter the data for the selected year
    df_selected_year = df[df['year_rept'] == year]

    # Calculate the crime frequency per area
    crime_frequency = df_selected_year['area_name'].value_counts().reset_index()
    crime_frequency.columns = ['area_name', 'crime_freq']

    # Calculate mean coordinates for each area
    if 'lat' in df_selected_year.columns and 'lon' in df_selected_year.columns:
        mean_coords = df_selected_year.groupby('area_name')[['lat', 'lon']].mean().reset_index()
        crime_frequency = pd.merge(crime_frequency, mean_coords, on='area_name', how='left')

    # Create a map centered on Los Angeles
    map_la = folium.Map(location=[34.0522, -118.2437], zoom_start=10, tiles='cartodbpositron')

    # Add a bubble/circle for each area
    for _, row in crime_frequency.iterrows():
        folium.CircleMarker(
            location=[row['lat'], row['lon']],
            radius=row['crime_freq'] * 0.0006,  # Scale the circles' radius
            color='blue',
            fill=True,
            fill_color='blue',
            fill_opacity=0.7,
            tooltip=f"Area: {row['area_name']}, Crime Frequency: {row['crime_freq']}"
        ).add_to(map_la)

    # Display the map
    display(map_la)

# Display the dropdown widget
display(year_dropdown)

# Display the map for the initial year
update_map(year_dropdown.value)

# Event handler for dropdown change
def on_dropdown_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        update_map(change['new'])

# Attach the event handler to the dropdown
year_dropdown.observe(on_dropdown_change)


In [ ]:
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster

# Load the data
data = pd.read_csv("Database/final/crime_data_modified.csv")

# Aggregate data to get crime frequency per area, and calculate mean latitude and longitude
data_aggregated = data.groupby('area_name').agg({
    'lat': 'mean',
    'lon': 'mean',
    'area_name': 'size'
}).rename(columns={'lat': 'avg_lat', 'lon': 'avg_lon', 'area_name': 'crime_count'}).reset_index()

# Load GeoJSON using geopandas (for boundaries, might be used in map visualization context)
la_geojson = gpd.read_file("Geospatial/los-angeles-county.geojson")

# Create the map with crime icons
m = folium.Map(location=[34.0522, -118.2437], zoom_start=10)  # Adjusted to center on Los Angeles

# Add marker cluster to map
marker_cluster = MarkerCluster().add_to(m)

# Define a color function
def color_producer(count):
    if count > data_aggregated['crime_count'].median():
        return 'red'
    else:
        return 'orange'

# Adding markers
for idx, row in data_aggregated.iterrows():
    folium.Marker(
        location=[row['avg_lat'], row['avg_lon']],
        popup=f"{row['area_name']} Crime count: {row['crime_count']}",
        icon=folium.Icon(color=color_producer(row['crime_count']), icon='exclamation-sign', prefix='glyphicon')
    ).add_to(marker_cluster)

# Display the map
m

In [ ]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster

# Load the data
data = pd.read_csv("Database/final/crime_data_modified_final.csv")

# Aggregate data for mean positions and crime counts
data_aggregated = data.groupby('area_name').agg({
    'lat': 'mean',
    'lon': 'mean',
    'area_name': 'size'
}).rename(columns={'lat': 'avg_lat', 'lon': 'avg_lon', 'area_name': 'crime_count'}).reset_index()

# Create the map centered on Los Angeles
m = folium.Map(location=[34.0522, -118.2437], zoom_start=10)

# Function to determine marker color based on crime count
def color_producer(count):
    if count > data_aggregated['crime_count'].median():
        return 'red'
    else:
        return 'orange'

# Adding main markers for average positions
for idx, row in data_aggregated.iterrows():
    # Main marker for the area average
    marker = folium.Marker(
        location=[row['avg_lat'], row['avg_lon']],
        popup=f"{row['area_name']} Crime count: {row['crime_count']}",
        icon=folium.Icon(color=color_producer(row['crime_count']), icon='star')
    )
    marker.add_to(m)

    # Nested Marker Cluster for individual crimes within the area
    nested_marker_cluster = MarkerCluster().add_to(marker)

    # Filter original data for the current area
    area_data = data[data['area_name'] == row['area_name']]
    for _, crime in area_data.iterrows():
        folium.Marker(
            location=[crime['lat'], crime['lon']],
            icon=folium.Icon(color='blue', icon='record', prefix='glyphicon'),
            popup=f"Crime: {crime['crime_type']}"
        ).add_to(nested_marker_cluster)

# Display the map
m

In [ ]:
import pandas as pd
import plotly.express as px

# Load the crime data
df = pd.read_csv("Database/final/crime_data_modified.csv")

# Convert 'date_rept' to datetime and extract the year
df['year_rept'] = pd.to_datetime(df['date_rept']).dt.year

# Filter for 'THEFT OF IDENTITY'
df_theft = df[df['cr_cd_desc'].str.upper() == 'THEFT OF IDENTITY']

# Calculate the crime frequency per area for each year
crime_rate = df_theft.groupby(['year_rept', 'area_name']).size().reset_index(name='crime_freq')

# Normalize by total crimes per year
total_crimes_per_year = df_theft.groupby('year_rept').size().reset_index(name='total_crimes')
crime_rate = pd.merge(crime_rate, total_crimes_per_year, on='year_rept')
crime_rate['crime_rate'] = crime_rate['crime_freq'] / crime_rate['total_crimes']

# Calculate mean coordinates for each area
mean_coords = df_theft.groupby('area_name')[['lat', 'lon']].mean().reset_index()
crime_rate = pd.merge(crime_rate, mean_coords, on='area_name', how='left')

# Plot using Plotly Express
fig = px.scatter_geo(crime_rate,
                     lat='lat',
                     lon='lon',
                     color="crime_rate",
                     size='crime_rate',
                     hover_name='area_name',
                     hover_data={'year_rept': True, 'crime_rate': True, 'area_name': False},
                     animation_frame='year_rept',
                     projection='natural earth',
                     title='THEFT OF IDENTITY Crime Rate by Area in Los Angeles Over Years',
                     template='plotly_dark')

fig.update_geos(fitbounds="locations", visible=False)
fig.show()


# Scaling factor for bubble size
scale_factor = 1000  # Adjust this value as needed to get the desired bubble size

# Plot using Plotly Express
fig = px.scatter_geo(crime_rate,
                     lat='lat',
                     lon='lon',
                     color="crime_rate",
                     size=crime_rate['crime_rate'] * scale_factor,  # Apply scaling factor here
                     hover_name='area_name',
                     hover_data={'year_rept': True, 'crime_rate': True, 'area_name': False},
                     animation_frame='year_rept',  # Ensure correct column name for animation
                     projection='natural earth',
                     title='THEFT OF IDENTITY Crime Rate by Area in Los Angeles Over Years',
                     template='plotly_dark')

fig.update_geos(fitbounds="locations", visible=False)
fig.show()



In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load the data from a CSV file
df = pd.read_csv('crime_data_final.csv')

# Ensure the date columns are in datetime format
df['date_rept'] = pd.to_datetime(df['date_rept'], errors='coerce')
df['date_occ'] = pd.to_datetime(df['date_occ'], errors='coerce')

# Calculate the delay in days between the reporting and the occurrence
df['delay_in_days'] = (df['date_rept'] - df['date_occ']).dt.days

# Filter out negative or unrealistic delays
df = df[df['delay_in_days'] >= 0]

# Create a violin plot
plt.figure(figsize=(12, 8))
sns.violinplot(x='area_name', y='delay_in_days', data=df)
plt.xticks(rotation=90)
plt.title('Distribution of Delays by Area')
plt.xlabel('Area')
plt.ylabel('Delay in Days')

# Show the plot
plt.tight_layout()
plt.show()

Sankey Diagram to Show Flow of Time

In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Load the data
df = pd.read_csv('Database/final/crime_data_modified_final.csv')

# Convert date columns to datetime
df['date_rept'] = pd.to_datetime(df['date_rept'])
df['date_occ'] = pd.to_datetime(df['date_occ'])

# Prepare the data by categorizing the delay into intervals
df['delay_hours'] = ((df['date_rept'] - df['date_occ']).dt.total_seconds() / 3600).astype(int)
df['delay_category'] = pd.cut(df['delay_hours'], bins=[0, 6, 12, 24, 48, 72, 168], labels=['0-6', '6-12', '12-24', '24-48', '48-72', '72-168'])

# Count occurrences in each category
delay_counts = df['delay_category'].value_counts().reset_index()
delay_counts.columns = ['delay_category', 'count']

# Create the Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=delay_counts['delay_category'],
    ),
    link=dict(
        source=[0, 1, 2, 3, 4],  # indices correspond to labels
        target=[1, 2, 3, 4, 5],
        value=delay_counts['count']
    ))])

fig.update_layout(title_text='Flow of Time from Occurrence to Reporting', font_size=10)
fig.show()

Temporal Heatmap with Time of Day

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load the data
df = pd.read_csv('Database/final/crime_data_modified_final.csv')

# Convert times to datetime and extract hour and day
df['date_occ'] = pd.to_datetime(df['date_occ'])
df['date_rept'] = pd.to_datetime(df['date_rept'])
df['hour'] = df['date_occ'].dt.hour
df['weekday'] = df['date_occ'].dt.day_name()

# Calculate the delay in hours
df['delay_hours'] = (df['date_rept'] - df['date_occ']).dt.total_seconds() / 3600

# Pivot table to organize the data
pivot_table = df.pivot_table(values='delay_hours', index='weekday', columns='hour', aggfunc='mean')

# Create the heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(pivot_table, cmap='viridis')
plt.title('Average Delay in Reporting (Hours) by Time of Day and Day of Week')
plt.xlabel('Hour of Day')
plt.ylabel('Day of Week')
plt.show()


In [ ]:
pip install altair